# 位置列表 

In [76]:
class DoublyLinkedBase():
    
    class _Node():
        __slots__ = '_elem', '_prev', '_next'
        def __init__(self, elem, prev, next):
            self._elem = elem
            self._prev = prev
            self._next = next
            
    def __init__(self):
        self._header = self._Node(None, None, None)
        self._trailer = self._Node(None, None, None)
        self._header._next = self._trailer
        self._trailer._prev = self._header
        self._size = 0
        
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0
    
    def _insert_between(self, e, predecessor, successor):
        """insert a node with element e between two nodes: pred and succ
        
        Params
        ======
        e: (any object) 
             reprsent a data
        predecessor: (self._Node)
        successor: (self._Node)   
        
        Returns
        =======
        (self._Node)
        """
        new = self._Node(e, predecessor, successor)
        predecessor._next = new
        successor._prev = new
        self._size += 1
        return new
        
    def _delete_node(self, node):
        predecessor = node._prev
        successor = node._next
        predecessor._next = successor
        successor._prev = predecessor
        self._size -= 1
        return node._elem

In [77]:
class PositionalList(DoublyLinkedBase):
    
    class Position():
        def __init__(self, container, node):
            self._container = container
            self._node = node
            
        def elem(self):
            return self._node._elem
        
        def __eq__(self, other):
            return type(self) == type(other) and self._node == other._node
        
        def __ne__(self, other):
            return not (self == other)
        
    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError('p must be a position!')
        if p._container is not self:
            raise ValueError('p should belong to this container!')
        if p._node._next is None or p._node._prev is None:
            raise ValueError('p is invalid')
        return p._node
    
    def _make_position(self, node):
        if node is self._header or node is self._trailer:
            return None
        return self.Position(self, node)
    
    def first(self):
        return self._make_position(self._header._next)
    
    def last(self):
        return self._make_position(self._trailer._prev)
    
    def before(self, p):
        node = self._validate(p)
        return self._make_position(node._prev)
    
    def after(self, p):
        node = self._validate(p)
        return self._make_position(node._next)
    
    def _insert_between(self, e, pred, succ):
        node = super()._insert_between(e, pred, succ)
        return self._make_position(node)
    
    def add_first(self, e):
        return self._insert_between(e, self._header, self._header._next)
    
    def add_last(self, e):
        return self._insert_between(e, self._trailer._prev, self._trailer)
    
    def add_before(self, p, e):
        node = self._validate(p)
        return self._insert_between(e, node._prev, node)
    
    def add_after(self, p, e):
        node = self._validate(p)
        return self._insert_between(e, node, node._next)
    
    def delete(self, p):
        node = self._validate(p)
        return self._delete_node(node)
      
    def replace(self, p, e):
        node = self._validate(p)
        old = node._elem
        node._elem = e
        return old
    
    def __iter__(self):
        cursor = self.first()
        while cursor is not None:
            yield cursor.elem()
            cursor = self.after(cursor)
        
    def __repr__(self):
        return f'{list(iter(self))}'  
    
    

In [78]:
L = PositionalList()
print(L)

p = L.add_last(8)

q = L.add_after(p, 5)
print(L)

r = L.add_before(q, 3)
print(L)

s = L.add_first(9)
print(L)

old = L.replace(r, 11)
print(old, L)

[]
[8, 5]
[8, 3, 5]
[9, 8, 3, 5]
3 [9, 8, 11, 5]
